<a href="https://colab.research.google.com/github/hamidb201214-svg/Lectures/blob/main/FlowiseAI_Docker_and_Streamlit_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FlowiseAI Framework: Docker + Flowise Container (Port 3000) + Streamlit Notes

This notebook packages a simple, copy/paste-friendly walkthrough for:

- Installing **Docker Desktop** on **macOS** and **Windows**
- Pulling and running **`flowiseai/flowise:latest`** on **port 3000**
- Quick notes for running a **Streamlit** app (locally or alongside Flowise)

> Tip: In Jupyter, lines starting with `!` run a shell command.


## Step 1 — Install Docker

### macOS

1. **Download Docker Desktop for Mac**
   - Search for: **“Docker Desktop for Mac”** (from Docker’s official site)
   - Choose the correct build for **Apple Silicon** or **Intel**

2. **Install Docker Desktop**
   - Open the downloaded **.dmg**
   - Drag the **Docker** icon into **Applications**

3. **Start Docker**
   - Open **Docker Desktop** from **Applications**
   - Follow the on-screen setup prompts

4. **Verify installation**
   - Open **Terminal** and run:

```
docker --version
```


### Windows

1. **Download Docker Desktop for Windows**
   - Search for: **“Docker Desktop for Windows”** (from Docker’s official site)

2. **Install Docker Desktop**
   - Run the installer
   - Ensure **WSL 2 components** are enabled/installed when prompted

3. **Start Docker**
   - Open **Docker Desktop**
   - Follow the setup prompts

4. **Verify installation**
   - Open **PowerShell** or **Command Prompt** and run:

```
docker --version
```


## Step 2 — Pull and Run `flowiseai/flowise:latest` on Port 3000

### 2.1 Pull the Docker image

Run:


In [ ]:
!docker pull flowiseai/flowise:latest

### 2.2 Run the container on port 3000

Run:

- `-d` runs in **detached** (background) mode
- `-p 3000:3000` maps your machine’s port **3000** to the container’s port **3000**


In [ ]:
!docker run -d --name flowise -p 3000:3000 flowiseai/flowise:latest

### 2.3 Check that it’s running (optional)

In [ ]:
!docker ps

### 2.4 Access Flowise

Open your browser to:

```
http://localhost:3000
```


## Helpful Docker Commands (Optional)

Stop / start / remove the Flowise container:


In [ ]:
# Stop the container
!docker stop flowise

# Start it again
!docker start flowise

# View logs
!docker logs -f flowise

# Remove container (must be stopped first)
# !docker rm flowise

## Streamlit App (Quick Notes)

If you also want to run a **Streamlit** app locally (separate from Flowise):

1. Install Streamlit
2. Create a simple `app.py`
3. Run `streamlit run app.py`

By default Streamlit uses port **8501**.


In [ ]:
# 1) Install Streamlit (run once)
!python -m pip install --upgrade streamlit

### Example `app.py`

This cell writes a tiny Streamlit app file you can run right away.


In [1]:
import streamlit as st
import requests
from requests.exceptions import RequestException

# --- Flowise API ---
API_URL = "http://localhost:3000/api/v1/prediction/7754964c-3236-4347-982b-f7e6c69391ce"

def query_flowise(question: str):
    payload = {"question": question}
    resp = requests.post(API_URL, json=payload, timeout=60)
    resp.raise_for_status()
    return resp.json()

# --- Streamlit UI ---
st.set_page_config(page_title="Hello Streamlit", layout="centered")
st.title("Hello Streamlit 👋")
st.write("This app sends your question to Flowise and shows the response.")

name = st.text_input("Your name")
if name:
    st.success(f"Nice to meet you, {name}!")

st.divider()
st.subheader("Ask Flowise")

question = st.text_input("Your question", value="Hey, how are you?")

if st.button("Send to Flowise", type="primary", disabled=not question.strip()):
    with st.spinner("Querying Flowise..."):
        try:
            output = query_flowise(question.strip())

            # Many Flowise flows return text under keys like "text" or "answer".
            # We'll show a friendly view if found, otherwise show the full JSON.
            possible_keys = ["text", "answer", "result", "response"]
            found = next((k for k in possible_keys if isinstance(output, dict) and k in output), None)

            if found:
                st.markdown("**Flowise reply:**")
                st.write(output[found])
                st.caption(f"(Displayed from key: `{found}`)")
                st.expander("Full JSON response").json(output)
            else:
                st.markdown("**Flowise JSON response:**")
                st.json(output)

        except RequestException as e:
            st.error("Could not reach Flowise or the request failed.")
            st.code(str(e))
        except Exception as e:
            st.error("Unexpected error.")
            st.code(str(e))


ModuleNotFoundError: No module named 'streamlit'

### Run the Streamlit app

Run:

```
streamlit run app.py
```

Then open:

```
http://localhost:8501
```


In [ ]:
# Start Streamlit (this will run until you interrupt the cell)
# !streamlit run app.py

## Running Flowise + Streamlit Together

- Flowise: `http://localhost:3000`
- Streamlit: `http://localhost:8501`

They can run at the same time because they use different ports.
